In [ ]:
!pip install aicrowd-cli
%load_ext aicrowd.magic

     |████████████████████████████████| 44 kB 1.4 MB/s 
     |████████████████████████████████| 54 kB 1.9 MB/s 
     |████████████████████████████████| 1.1 MB 47.0 MB/s 
     |████████████████████████████████| 170 kB 62.9 MB/s 
     |████████████████████████████████| 214 kB 64.4 MB/s 
     |████████████████████████████████| 62 kB 886 kB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 51 kB 6.5 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: pyzmq
    Found existing installation: pyzmq 22.3.0
    Uninstalling pyzmq-22.3.0:
      Successfully uninstalled pyzmq-22.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you hav

In [ ]:
%aicrowd login

Please login here: https://api.aicrowd.com/auth/jrkPj0d4QdnvF0sEyoxwfrrRYo7dET6AUXXIJ8QUAww
API Key valid
Saved API Key successfully!


In [ ]:
!rm -rf data
!mkdir data
%aicrowd ds dl -c embedding-game -o data

placeholder_vectors.csv:   0%|          | 0.00/599k [00:00<?, ?B/s]

sample_submission.csv:   0%|          | 0.00/5.86k [00:00<?, ?B/s]

test_words.txt:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

train_labels.csv:   0%|          | 0.00/507k [00:00<?, ?B/s]

In [ ]:
import pandas as pd
import numpy as np
import os
import json
import ast
from sklearn.metrics.pairwise import cosine_similarity

from gensim.models import Word2Vec
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
train_df = pd.read_csv("data/train_labels.csv")

In [ ]:
train_df.head()

,embeddings,label
0,"[0.525386, 0.61333424, 0.62004805, 0.5554309, ...",exchanges
1,"[0.5653091, 0.61592805, 0.54518753, 0.575984, ...",equines
2,"[0.69483936, 0.59534645, 0.5720794, 0.7992547,...",Was
3,"[0.59585017, 0.5326628, 0.6304897, 0.6168022, ...",systems
4,"[0.6212443, 0.53205186, 0.5439351, 0.5772528, ...",remainder


In [ ]:
train_df.shape

(149, 2)

In [ ]:
with open("data/test_words.txt", "r") as f:
  text = f.read()
test_words = text.split(',')
test_words

['equine',
 'is',
 'assist',
 'refers',
 'the',
 'progressive',
 'when',
 'divorce',
 'not',
 'allocate',
 'control',
 'they',
 'and',
 'lawsuits',
 'should',
 'often',
 'in',
 'sums',
 'purchase',
 'associated',
 'some',
 'gambling',
 'valuation',
 'prepared',
 'revenue',
 'establish',
 'contingent',
 'sell',
 'outlay',
 'other',
 'exchange',
 'have',
 'relation',
 'taking',
 'related',
 'addition',
 'growing',
 'agreements',
 'shareholders',
 'pay',
 'accounting',
 'longer',
 'high',
 'sector',
 'estimating',
 'partners',
 'such',
 'industry',
 'disastrous',
 'stock',
 'among',
 'structure',
 'could',
 'lost',
 'action',
 'might',
 'above',
 'prices',
 'she',
 'profit',
 'goods',
 'services',
 'company',
 'misusing',
 'has',
 'estate',
 'force',
 'by',
 'consume',
 'within',
 'place',
 'subsidize',
 'resolve',
 'disputes',
 'initial',
 'value',
 'legal',
 'deadlock',
 'information',
 'dollars',
 'farm',
 'formula',
 'contest',
 'buy',
 'performance',
 'further',
 'many',
 'finance',


In [ ]:
len(test_words)

181

In [ ]:
train_df.embeddings = train_df.embeddings.apply(lambda x: ast.literal_eval(x))

In [ ]:
len(train_df['embeddings'][0])

300

In [ ]:
train_df.head(5)

,embeddings,label
0,"[0.525386, 0.61333424, 0.62004805, 0.5554309, ...",exchanges
1,"[0.5653091, 0.61592805, 0.54518753, 0.575984, ...",equines
2,"[0.69483936, 0.59534645, 0.5720794, 0.7992547,...",Was
3,"[0.59585017, 0.5326628, 0.6304897, 0.6168022, ...",systems
4,"[0.6212443, 0.53205186, 0.5439351, 0.5772528, ...",remainder


In [ ]:
col=[]
for i in range(300):
  s='Col-'+str(i)
  col.append(s)

In [ ]:
rows=[]
for i in train_df['embeddings']:
  rows.append(i)

In [ ]:
len(col),len(rows)

(300, 149)

In [ ]:
len(rows[0])

300

In [ ]:
df = pd.DataFrame(rows,
                  columns = col)

In [ ]:
df.shape

(149, 300)

In [ ]:
df.head(5)

,Col-0,Col-1,Col-2,Col-3,Col-4,Col-5,Col-6,Col-7,Col-8,Col-9,Col-10,Col-11,Col-12,Col-13,Col-14,Col-15,Col-16,Col-17,Col-18,Col-19,Col-20,Col-21,Col-22,Col-23,Col-24,Col-25,Col-26,Col-27,Col-28,Col-29,Col-30,Col-31,Col-32,Col-33,Col-34,Col-35,Col-36,Col-37,Col-38,Col-39,...,Col-260,Col-261,Col-262,Col-263,Col-264,Col-265,Col-266,Col-267,Col-268,Col-269,Col-270,Col-271,Col-272,Col-273,Col-274,Col-275,Col-276,Col-277,Col-278,Col-279,Col-280,Col-281,Col-282,Col-283,Col-284,Col-285,Col-286,Col-287,Col-288,Col-289,Col-290,Col-291,Col-292,Col-293,Col-294,Col-295,Col-296,Col-297,Col-298,Col-299
0,0.525386,0.613334,0.620048,0.555431,0.669389,0.524727,0.526745,0.589699,0.649061,0.663398,0.527262,0.527446,0.525798,0.765231,0.536563,0.578431,0.586390,0.747255,0.557703,0.530900,0.604994,0.553676,0.689884,0.583744,0.543629,0.584657,0.525333,0.628602,0.698191,0.665628,0.525555,0.524353,0.551534,0.568298,0.595095,0.548712,0.609663,0.568538,0.648368,0.594345,...,0.560364,0.637580,0.570981,0.625499,0.558199,0.706738,0.703290,0.562028,0.529971,0.646302,0.524536,0.550659,0.540931,0.552350,0.586390,0.539076,0.540931,0.597373,0.564597,0.792671,0.607722,0.644936,0.567581,0.575565,0.652561,0.540550,0.525041,0.684210,0.661924,0.556369,0.647677,0.524807,0.643578,0.597885,0.603382,0.633037,0.681019,0.581608,0.533708,0.677867
1,0.565309,0.615928,0.545188,0.575984,0.573396,0.599433,0.559977,0.687438,0.584200,0.540911,0.613907,0.591357,0.527353,0.524843,0.581385,0.547481,0.533298,1.150222,0.533875,0.726398,0.679438,0.535251,0.585578,0.530699,0.526331,0.590904,0.534071,0.737624,0.606079,0.540193,0.524498,0.571578,0.671673,0.570832,0.525333,0.784040,0.526353,0.529530,0.576828,0.644936,...,0.524377,0.618270,0.695674,0.595095,0.627354,0.524299,0.542963,0.528134,0.533226,0.621846,0.525555,0.790497,0.528309,0.640224,0.567915,0.529971,0.562926,0.568106,0.524761,0.828896,0.653267,0.531394,0.625499,0.543446,0.530105,0.561393,0.544744,0.524350,0.857886,1.027500,0.531181,0.588269,0.529851,0.553715,0.607447,0.701581,0.737624,0.743373,0.581942,0.643578
2,0.694839,0.595346,0.572079,0.799255,0.577466,0.545571,0.535997,0.541384,0.638237,0.577787,0.528560,0.524319,0.528560,0.530564,0.726398,0.590179,0.525333,0.683409,0.530972,0.542135,0.561808,0.733842,0.792671,0.536948,0.568250,0.589220,0.527580,0.667877,0.627354,0.728244,0.567391,0.531041,0.527930,0.805934,0.569701,0.673207,0.585116,0.579952,0.570044,0.659002,...,0.707606,0.544556,0.620645,0.577253,0.529009,1.014864,0.534819,0.524454,0.528560,0.611346,0.524536,0.574836,0.775576,0.651856,0.649061,0.948821,0.525181,0.524410,0.662660,0.549481,0.619156,0.524954,0.527580,0.564666,0.559835,0.619453,0.526766,0.524382,0.613620,0.568972,0.543918,0.730100,0.524706,0.831252,0.547279,0.526827,0.528134,0.530662,0.575044,0.654687
3,0.595850,0.532663,0.630490,0.616802,0.524350,0.595346,0.544493,0.616802,0.564506,0.549551,0.532127,0.535555,0.560539,0.556632,0.551241,0.550985,0.558789,0.548850,0.547955,0.564872,0.552164,0.556309,0.564369,0.611064,0.611064,0.592367,0.525798,0.592122,0.544306,0.524675,0.574732,0.524548,0.535536,0.545443,0.571829,0.524454,0.561436,0.531111,0.615928,0.542370,...,0.566918,0.525333,0.606079,0.649757,0.594345,0.565795,0.542018,0.544120,0.529971,0.697349,0.530699,0.528502,0.543325,0.616510,0.545124,0.539395,0.565517,0.552952,0.554918,0.562659,0.527241,0.646302,0.622449,0.569653,0.540818,0.540153,0.697349,0.540043,0.553715,0.580721,0.524410,0.634971,0.554957,0.686627,0.608274,0.545188,0.556390,0.525674,0.562382,0.556289
4,0.621244,0.532052,0.543935,0.577253,0.597118,0.525333,0.538554,0.529851,0.662660,0.526827,0.524382,0.531977,0.550876,0.658277,0.524289,0.528134,0.540373,0.565424,0.547955,0.615348,0.550623,0.574010,0.545700,0.638897,0.628602,0.677084,0.529726,0.620645,0.544997,0.719111,0.524727,0.532278,0.581719,0.641559,0.623055,0.608551,0.525000,0.524367,0.549129,0.531977,...,0.606625,0.601525,0.526195,0.624272,0.544493,0.578108,0.542370,0.622449,0.602849,0.560587,0.636271,0.527408,0.557590,0.524498,0.587325,

In [ ]:
X=df.iloc[:]

In [ ]:
Y=train_df.iloc[:,1]

In [ ]:
X=rows

In [ ]:
Y=[c.lower() for c in Y]

In [ ]:
Y=[c.replace('_','') for c in Y]

In [ ]:
len(Y),len(X)

(149, 149)

In [ ]:
for i in range(len(X)):
  print(max(X[i]),min(X[i]))

##SUBMISSION

In [ ]:
test_df=pd.read_csv('data/placeholder_vectors.csv')

In [ ]:
X_test=[]
for i in test_df.embeddings:
  try:
    j=ast.literal_eval(i)
    X_test.append(j)
  except:
    print(i)

a
of
to
and


In [ ]:
test_df.head()

,placeholder,embeddings
0,0,"[0.72639763, 0.55363804, 0.5709315, 0.69988114..."
1,1,"[0.570044, 0.53571916, 0.5704866, 0.6747511, 0..."
2,2,a
3,3,"[0.5926122, 0.56348646, 0.5378386, 0.65755475,..."
4,4,"[0.6739781, 0.5331372, 0.5242991, 0.5666829, 0..."


In [ ]:
from numpy import dot
from numpy.linalg import norm
def similarity(l1,l2):
  cos_sim = dot(l1, l2)/(norm(l1)*norm(l2))
  return cos_sim

In [ ]:
def custom_similarity(l1,l2):
  index=1
  similar=0
  for i in range(len(l1)):
    j=(l1[i]-l2[i])*(l1[i]-l2[i])
    similar=similar+(j*index)
    index+=1
  return similar

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
def get_cosine_similarity(feature_vec_1, feature_vec_2):
  return cosine_similarity(feature_vec_1, feature_vec_2)[0][0]

In [ ]:
get_cosine_similarity(['remainder'],['return'])

╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ /usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882 in run_code  │
│                                                                                           │
│   2879 │   │   │   try:                                                                   │
│   2880 │   │   │   │   self.hooks.pre_run_code_hook()                                     │
│   2881 │   │   │   │   #rprint('Running code', repr(code_obj)) # dbg                      │
│ ❱ 2882 │   │   │   │   exec(code_obj, self.user_global_ns, self.user_ns)                  │
│   2883 │   │   │   finally:                                                               │
│   2884 │   │   │   │   # Reset our crash handler in place                                 │
│   2885 │   │   │   │   sys.excepthook = old_excepthook                                    │
│ <ipython-input-256-522a24770447>:1 in <module>                                            │
│ <ipython-input-255-a2cff6124cc5>:3 in get_cosine_similarity                               │
│                                                                                           │
│ /usr/local/lib/python3.7/dist-packages/sklearn/metrics/pairwise.py:1246 in                │
│ cosine_similarity                                                                         │
│                                                                                           │
│   1243 │   """                                                                            │
│   1244 │   # to avoid recursive import                                                    │
│   1245 │                                                                                  │
│ ❱ 1246 │   X, Y = check_pairwise_arrays(X, Y)                                             │
│   1247 │                                                                                  │
│   1248 │   X_normalized = normalize(X, copy=True)                                         │
│   1249 │   if X is Y:                                                                     │
│                                                                                           │
│ /usr/local/lib/python3.7/dist-packages/sklearn/metrics/pairwise.py:162 in                 │
│ check_pairwise_arrays                                                                     │
│                                                                                           │
│    159 │   │   │   dtype=dtype,                                                           │
│    160 │   │   │   copy=copy,                                                             │
│    161 │   │   │   force_all_finite=force_all_finite,                                     │
│ ❱  162 │   │   │   estimator=estimator,                                                   │
│    163 │   │   )                                                                          │
│    164 │   │   Y = check_array(                                                           │
│    165 │   │   │   Y,                                                                     │
│                                                                                           │
│ /usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:738 in check_array     │
│                                                                                           │
│    735 │   │   │   │   │   │   _assert_all_finite(array, allow_nan=False, msg_dtype=dtype │
│    736 │   │   │   │   │   array = array.astype(dtype, casting="unsafe", copy=False)      │
│    737 │   │   │   │   else:                                                              │
│ ❱  738 │   │   │   │   │   array = np.asarray(array, order=order, dtype=dtype)            │
│    739 │   │   │   except ComplexWarning as comple

In [ ]:
from scipy.spatial import distance
def euclidean_similarity(a,b):
  return distance.euclidean(a, b)

In [ ]:
custom_similarity(X_test[0],X[0])

387.57544130693526

In [ ]:
import string
count=0
rest=0
d={}
Y_new=[]
for i in Y:
  j=i.lower()
  line = ''.join(c for c in j if c in string.ascii_letters)
  Y_new.append(line)
  if line not in test_words:
    f=False
    for k in test_words:
      for l in test_words:
        if k!=l and (k+l==j or l+k==line) and k!='s' and l!='s':
          d[line]=[k,l]
          f=True
          rest+=1
          print(k,l,line)
          #print(line,k,l)
          break
      if f==True:
        break
    if f==False:
      count+=1
      print(line,j)

exchanges exchanges
equines equines
systems systems
remainder remainder
amongst amongst
decisions decisions
the from fromthe
well well
dispute dispute
gifts gifts
partnership partnership
the on onthe
individuals individuals
outlays outlays
subsidy subsidy
risks risks
the by bythe
allocating allocating
actions actions
these these
vital vital
never never
thousanddollars thousanddollars
resolving resolving
losing losing
frequently frequently
may may
assisting assisting
contingents contingents
several several
martin martinâ€™
tool tool
the into intothe
appraiser appraiser
complicated complicated
year year
managing managing
function function
taxes taxes
havent havent
stalemate stalemate
either either
oversubscribed oversubscribed
agreement agreement
service service
ventures ventures
private private
estates estates
farms farms
investments investments
divorces divorces
valuations valuations
purpose purpose
relating relating
ownerships ownerships
times times
places places
shareholder sharehold

In [ ]:
Y=Y_new

In [ ]:
Y[0]

'exchanges'

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

stemmer = nltk.stem.porter.PorterStemmer()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)

def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]

'''remove punctuation, lowercase, stem'''
def normalize(text):
    return stem_tokens(nltk.word_tokenize(text.lower().translate(remove_punctuation_map)))

vectorizer = TfidfVectorizer(tokenizer=normalize)

def cosine_sim(text1, text2):
    tfidf = vectorizer.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0,1]

In [ ]:
cosine_sim('gen_eral','general')

1.0

In [ ]:
dd={}
rest=[]
for i in Y:
  l=[]
  if i in d:
    i=' '.join(d[i])
  #print(i)
  for j in test_words:
    l.append(cosine_sim(i,j))
  k=np.argmax(l)
  if k==0:
    rest.append(i)
    continue
  ll=[]
  for j in range(len(l)):
    if l[j]==l[k]:
      ll.append(test_words[j])
  dd[i]=ll

In [ ]:
len(dd),len(Y),len(rest)

(102, 149, 47)

In [ ]:
l=[]
for j in test_words:
  l.append(cosine_sim(j,rest[1]))
print(rest[1],l)

remainder [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [ ]:
!pip install spacy-udpipe

     |████████████████████████████████| 6.0 MB 12.0 MB/s 
     |████████████████████████████████| 304 kB 35.5 MB/s 
     |████████████████████████████████| 628 kB 49.8 MB/s 
     |████████████████████████████████| 451 kB 51.6 MB/s 
     |████████████████████████████████| 42 kB 1.2 MB/s 
     |████████████████████████████████| 10.1 MB 47.6 MB/s 
     |████████████████████████████████| 181 kB 59.6 MB/s 
  Created wheel for ufal.udpipe: filename=ufal.udpipe-1.2.0.3-cp37-cp37m-linux_x86_64.whl size=5626606 sha256=8b2913f31820bac5827be49012979583018b1ae77e5f677e93b2e5752db73ef9
  Stored in directory: /root/.cache/pip/wheels/b8/b5/8e/3da091629a21ce2d10bf90759d0cb034ba10a5cf7a01e83d64
Successfully built ufal.udpipe
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Success

In [ ]:
!pip install ufal.udpipe

In [ ]:
# in Python: These libraries are getting loaded. 
import spacy 

import en_core_web_sm

In [ ]:
python -m download en_core_web_sm

╭───────────────────────────────────────────────────────────────────────────────────────────╮
│  <ipython-input-276-c5140aba1463>:1                                                       │
│ python -m download en_core_web_sm                                                         │
│                  ▲                                                                        │
╰───────────────────────────────────────────────────────────────────────────────────────────╯
SyntaxError: invalid syntax

In [ ]:
import spacy_udpipe

spacy_udpipe.download("italian-postwita")

╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ /usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882 in run_code  │
│                                                                                           │
│   2879 │   │   │   try:                                                                   │
│   2880 │   │   │   │   self.hooks.pre_run_code_hook()                                     │
│   2881 │   │   │   │   #rprint('Running code', repr(code_obj)) # dbg                      │
│ ❱ 2882 │   │   │   │   exec(code_obj, self.user_global_ns, self.user_ns)                  │
│   2883 │   │   │   finally:                                                               │
│   2884 │   │   │   │   # Reset our crash handler in place                                 │
│   2885 │   │   │   │   sys.excepthook = old_excepthook                                    │
│ <ipython-input-271-10b8426201bf>:1 in <module>                                            │
│                                                                                           │
│ /usr/local/lib/python3.7/dist-packages/spacy_udpipe/__init__.py:7 in <module>             │
│                                                                                           │
│    4 │   "UDPipeTokenizer", "UDPipeModel"                                                 │
│    5 ]                                                                                    │
│    6                                                                                      │
│ ❱  7 from .utils import download, load, load_from_path                                    │
│    8 from .tokenizer import UDPipeTokenizer                                               │
│    9 from .udpipe import UDPipeModel                                                      │
│   10                                                                                      │
│                                                                                           │
│ /usr/local/lib/python3.7/dist-packages/spacy_udpipe/utils.py:6 in <module>                │
│                                                                                           │
│     3 import urllib.request                                                               │
│     4 from typing import Optional, Dict                                                   │
│     5                                                                                     │
│ ❱   6 from spacy import blank, Language                                                   │
│     7 from spacy.util import get_lang_class                                               │
│     8                                                                                     │
│     9                                                                                     │
╰───────────────────────────────────────────────────────────────────────────────────────────╯
ImportError: cannot import name 'Language' from 'spacy' 
(/usr/local/lib/python3.7/dist-packages/spacy/__init__.py)

In [ ]:
import spacy
nlp = spacy.load('en_core_web_md')

╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ /usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882 in run_code  │
│                                                                                           │
│   2879 │   │   │   try:                                                                   │
│   2880 │   │   │   │   self.hooks.pre_run_code_hook()                                     │
│   2881 │   │   │   │   #rprint('Running code', repr(code_obj)) # dbg                      │
│ ❱ 2882 │   │   │   │   exec(code_obj, self.user_global_ns, self.user_ns)                  │
│   2883 │   │   │   finally:                                                               │
│   2884 │   │   │   │   # Reset our crash handler in place                                 │
│   2885 │   │   │   │   sys.excepthook = old_excepthook                                    │
│ <ipython-input-277-b8962b0d445d>:3 in <module>                                            │
│                                                                                           │
│ /usr/local/lib/python3.7/dist-packages/spacy/__init__.py:30 in load                       │
│                                                                                           │
│   27 │   raise SystemError(Errors.E130)                                                   │
│   28                                                                                      │
│   29                                                                                      │
│ ❱ 30 def load(                                                                            │
│   31 │   name: Union[str, Path],                                                          │
│   32 │   *,                                                                               │
│   33 │   vocab: Union[Vocab, bool] = True,                                                │
│                                                                                           │
│ /usr/local/lib/python3.7/dist-packages/spacy/util.py:159 in load_model                    │
│                                                                                           │
│    156 │   │   reg = getattr(cls, registry_name)                                          │
│    157 │   │   try:                                                                       │
│    158 │   │   │   func_info = reg.find(func_name)                                        │
│ ❱  159 │   │   except RegistryError:                                                      │
│    160 │   │   │   if func_name.startswith("spacy."):                                     │
│    161 │   │   │   │   legacy_name = func_name.replace("spacy.", "spacy-legacy.")         │
│    162 │   │   │   │   try:                                                               │
╰───────────────────────────────────────────────────────────────────────────────────────────╯
OSError: [E049] Can't find spaCy data directory: 'None'. Check your installation and 
permissions, or use spacy.util.set_data_path to customise the location if necessary.

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
from nltk.corpus import wordnet

list1 = ['Compare', 'require']
list2 = ['choose', 'copy', 'define', 'duplicate', 'find', 'how', 'identify', 'label', 'list', 'listen', 'locate', 'match', 'memorise', 'name', 'observe', 'omit', 'quote', 'read', 'recall', 'recite', 'recognise', 'record', 'relate', 'remember', 'repeat', 'reproduce', 'retell', 'select', 'show', 'spell', 'state', 'tell', 'trace', 'write']
list = []

for word1 in list1:
    for word2 in list2:
        wordFromList1 = wordnet.synsets(word1)
        wordFromList2 = wordnet.synsets(word2)
        if wordFromList1 and wordFromList2:
            s = wordFromList1[0].wup_similarity(wordFromList2[0])
            list.append(s)

print(max(list))

╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ /usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882 in run_code  │
│                                                                                           │
│   2879 │   │   │   try:                                                                   │
│   2880 │   │   │   │   self.hooks.pre_run_code_hook()                                     │
│   2881 │   │   │   │   #rprint('Running code', repr(code_obj)) # dbg                      │
│ ❱ 2882 │   │   │   │   exec(code_obj, self.user_global_ns, self.user_ns)                  │
│   2883 │   │   │   finally:                                                               │
│   2884 │   │   │   │   # Reset our crash handler in place                                 │
│   2885 │   │   │   │   sys.excepthook = old_excepthook                                    │
│ <ipython-input-234-0eb8b321b89a>:15 in <module>                                           │
╰───────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: '>' not supported between instances of 'float' and 'NoneType'

In [ ]:
l=[]
ll=[]
for i in rest:
  print(i,l)
  for j in test_words:
    l.append(cosine_sim(j,'equines'))
  ll.append(test_words[np.argmax(l)])
  l.clear()

equines []
remainder []
amongst []
well []
partnership []
individuals []
subsidy []
these []
vital []
never []
thousanddollars []
losing []
frequently []
may []
several []
martin []
complicated []
havent []
stalemate []
either []
oversubscribed []
private []
hundreds []
below []
additon []
england []
larger []
firm []
sold []
managership []
grown []
themselves []
four []
attendees []
he []
ought []
low []
calamitous []
were []
less []
paid []
made []
this []
similar []
info []
repaid []
had []


In [ ]:
len(rest),ll[0]

(47, 'equine')

In [ ]:
for i in ll:
  if i not in ['equine']:
    print(i)

In [ ]:
rest_fill

{'additon': ['addition'],
 'amongst': ['among'],
 'attendees': ['at'],
 'below': ['be'],
 'calamitous': ['can', 'capital'],
 'complicated': ['complex'],
 'either': ['equine',
  'establish',
  'exchange',
  'estimating',
  'estate',
  'economy',
  'economic'],
 'england': ['equine',
  'establish',
  'exchange',
  'estimating',
  'estate',
  'economy',
  'economic'],
 'equines': ['equine'],
 'firm': ['finance'],
 'four': ['force', 'formula', 'for'],
 'frequently': ['from', 'friends'],
 'grown': ['growing'],
 'had': ['have', 'has'],
 'havent': ['have'],
 'he': ['have', 'high', 'has', 'horses', 'horse', 'his', 'him'],
 'hundreds': ['have', 'high', 'has', 'horses', 'horse', 'his', 'him'],
 'individuals': ['industry'],
 'info': ['information'],
 'larger': ['large'],
 'less': ['legal'],
 'losing': ['lost'],
 'low': ['longer', 'lost', 'london'],
 'made': ['many',
  'management',
  'making',
  'managerial',
  'managers',
  'market',
  'markets'],
 'managership': ['managers'],
 'martin': ['marke

In [ ]:
rest_fill

In [ ]:
rest_fills={}
for i in rrest:
  print(i)
  print('**')
  f=False
  k=i
  while(len(k)!=0):
    temp=[]
    for j in test_words:
      if j.startswith(k):
        f=True
        temp.append(j)
        print(j)
    if f==True:
      rest_fills[i]=temp
      break
    k=k[:len(k)-1]
  print('***')

well
**
when
within
with
was
***
partnership
**
partners
***
subsidy
**
subsidize
***
vital
**
valuation
value
venture
***
thousanddollars
**
thousands
***
martin
**
market
markets
***
havent
**
have
***
england
**
equine
establish
exchange
estimating
estate
economy
economic
***
themselves
**
the
they
their
***
similar
**
should
sums
some
sell
shareholders
sector
such
stock
structure
she
services
subsidize
s
selling
same
system
shares
subscribed
small
***


In [ ]:
rest_fill['well']='when'
rest_fill['partnership']='partners'
rest_fill['subsidy']='subsidize'
rest_fill['vital']='value'
rest_fill['martin']='market'
rest_fill['havent']='have'
rest_fill['england']='estate'
rest_fill['themselves']='their'
rest_fill['similar']='same'


In [ ]:

for i in rest_fill.keys():
  if(len(rest_fill[i])>1):
    print(i,rest_fill[i])

remainder ['refers', 'revenue', 'relation', 'related', 'resolve', 'repay', 'rest']
well ['when', 'within', 'with', 'was']
these ['the', 'they', 'their']
vital ['valuation', 'value', 'venture']
frequently ['from', 'friends']
may ['many', 'management', 'making', 'managerial', 'managers', 'market', 'markets']
several ['sell', 'sector', 'services', 'selling']
martin ['market', 'markets']
stalemate ['stock', 'structure']
either ['equine', 'establish', 'exchange', 'estimating', 'estate', 'economy', 'economic']
oversubscribed ['often', 'outlay', 'other', 'on', 'of', 'opinion', 'ownership', 'organizational', 'or']
private ['prices', 'price']
hundreds ['have', 'high', 'has', 'horses', 'horse', 'his', 'him']
england ['equine', 'establish', 'exchange', 'estimating', 'estate', 'economy', 'economic']
themselves ['the', 'they', 'their']
four ['force', 'formula', 'for']
he ['have', 'high', 'has', 'horses', 'horse', 'his', 'him']
low ['longer', 'lost', 'london']
calamitous ['can', 'capital']
were ['wh

In [ ]:

for i in rest:
  if i not in rest_fill.keys():
    print(i)

thousanddollars


In [ ]:
d['thousanddollars']

['thousands', 'dollars']

In [ ]:
for i in dd.values():
  if(len(i)==0):
    print(i)

In [ ]:
l=[]
for j in test_words:
  l.append(cosine_sim('bythe',j))
k=np.argmax(l)

In [ ]:
for i in rest_fill.keys():
  print(rest_fill[i])

equine
rest
among
people
many
not
lost
often
might
many
complex
deadlock
or
subscribed
public
thousands
above
addition
large
company
bought
management
growing
three
participants
she
should
high
disastrous
are
more
pay
was
the
information
repay
have
when
partners
subsidize
value
market
have
estate
their
same


In [ ]:
def func(embedding_list):
  similar=[]
  index=[]
  for i in range(len(Y)):
    #print(i)
    s=similarity(X[i],embedding_list)
    #print(s)
    similar.append(s)
    index.append(i)
  ll=sorted(zip(similar,index), reverse=True)[:4]
  #print(ll)
  lists=[]
  for i,j in ll:
    #print(Y[j])
    if Y[j] in d:
      for val in d[Y[j]]:
        if val not in lists:
          lists.append(val)
    elif Y[j] in dd:
      for val in dd[Y[j]]:
        if val not in lists:
          lists.append(val)
    else:
      if rest_fill[Y[j]] not in lists:
          lists.append(rest_fill[Y[j]])
  print(lists)
  return lists[:3]

In [ ]:
func(ast.literal_eval(test_df.embeddings[0]))

['the', 'capital', 'or']


['the', 'capital', 'or']

In [ ]:
arrays=[]
for i in test_df.embeddings:
  try:
    j=ast.literal_eval(i)
    arrays.append(func(j))
  except:
    listss=[]
    listss.append(i)
    listss.append(test_words[0])
    listss.append(test_words[1])
    arrays.append(listss)

['the', 'capital', 'or', 'she']
['the', 'on', 'or', 'within']
['high', 'the', 'when', 'capital']
['investment', 'tax', 'the', 'capital', 'general']
['should', 'might', 'or']
['pay', 'the', 'capital', 'market']
['more', 'the', 'or', 'capital']
['tax', 'subsidize', 'investment', 'general']
['public', 'general', 'the', 'capital', 'or']
['opinion', 'general', 'or', 'market']
['should', 'might', 'or', 'many']
['the', 'capital', 'force', 'she', 'or']
['the', 'within', 'or', 'capital']
['government', 'the', 'capital', 'or', 'general']
['the', 'into', 'in', 'or']
['the', 'or', 'market', 'capital']
['or', 'not', 'the', 'she']
['bought', 'have', 'the', 'capital']
['many', 'the', 'or']
['stock', 'the', 'capital', 'or', 'within']
['estate', 'the', 'capital', 'market', 'on']
['stock', 'the', 'capital', 'investment', 'or']
['the', 'capital', 'within', 'or', 'exchange']
['place', 'market', 'the', 'capital', 'within']
['bubble', 'or', 'the', 'when']
['the', 'might', 'or', 'within']
['type', 'the', 'or

In [ ]:
len(arrays),test_df.shape

(180, (180, 3))

In [ ]:
arrays[0]

['the', 'capital', 'or']

In [ ]:
predicted=[]
for i in arrays:
  j=str(i)
  #j=' '.join(c for c in i)
  #k='['+j+']'
  k=j.replace(',','')
  predicted.append(k)

In [ ]:
count=0
for i in arrays:
  for j in i:
    if j not in test_words:
      count+=1
      print(j)

In [ ]:
count

0

In [ ]:
predicted

["['the' 'capital' 'or']",
 "['the' 'on' 'or']",
 "['a' 'equine' 'is']",
 "['high' 'the' 'when']",
 "['investment' 'tax' 'the']",
 "['should' 'might' 'or']",
 "['pay' 'the' 'capital']",
 "['more' 'the' 'or']",
 "['tax' 'subsidize' 'investment']",
 "['public' 'general' 'the']",
 "['opinion' 'general' 'or']",
 "['should' 'might' 'or']",
 "['the' 'capital' 'force']",
 "['the' 'within' 'or']",
 "['government' 'the' 'capital']",
 "['the' 'into' 'in']",
 "['the' 'or' 'market']",
 "['or' 'not' 'the']",
 "['bought' 'have' 'the']",
 "['many' 'the' 'or']",
 "['stock' 'the' 'capital']",
 "['estate' 'the' 'capital']",
 "['stock' 'the' 'capital']",
 "['the' 'capital' 'within']",
 "['place' 'market' 'the']",
 "['bubble' 'or' 'the']",
 "['the' 'might' 'or']",
 "['type' 'the' 'or']",
 "['of' 'equine' 'is']",
 "['the' 'capital' 'general']",
 "['taking' 'the' 'or']",
 "['place' 'the' 'capital']",
 "['the' 'in' 'within']",
 "['place' 'market' 'the']",
 "['not' 'she' 'the']",
 "['participants' 'the' 'capi

In [ ]:
test_df['word'] = predicted

In [ ]:
type(test_df['word'][0])

str

In [ ]:
test_df.head(5)

,placeholder,embeddings,word
0,0,"[0.72639763, 0.55363804, 0.5709315, 0.69988114...",['the' 'capital' 'or']
1,1,"[0.570044, 0.53571916, 0.5704866, 0.6747511, 0...",['the' 'on' 'or']
2,2,a,['a' 'equine' 'is']
3,3,"[0.5926122, 0.56348646, 0.5378386, 0.65755475,...",['high' 'the' 'when']
4,4,"[0.6739781, 0.5331372, 0.5242991, 0.5666829, 0...",['investment' 'tax' 'the']


In [ ]:
for i in arrays:
  if len(i)!=3:
    print(i)

In [ ]:
!rm -rf assets
!mkdir assets
test_df.to_csv(os.path.join("assets", "submission.csv"))

In [ ]:
%aicrowd notebook submit -c embedding-game -a assets --no-verify

Using notebook: Embedding.ipynb for submission...
Removing existing files from submission directory...
Scrubbing API keys from the notebook...


Output()

╭─────────────────────────╮                                                  
                                                 │ Successfully submitted! │                                                  
                                                 ╰─────────────────────────╯

                                                       Important links                                                        
┌──────────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│  This submission │ https://www.aicrowd.com/challenges/ai-blitz-xii/problems/embedding-game/submissions/170359              │
│                  │                                                                                                         │
│  All submissions │ https://www.aicrowd.com/challenges/ai-blitz-xii/problems/embedding-game/submissions?my_submissions=true │
│                  │                                                                                                         │
│      Leaderboard │ https://www.aicrowd.com/challenges/ai-blitz-xii/problems/embedding-game/leaderboards                    │
│                  │                                                                                                         │
│ Discussion forum │ https://discourse.aicrowd.com/c/ai-blitz-xii                                                            │
│                  │                                                                                                         │
│   Challenge page │ https://www.aicrowd.com/challenges/ai-blitz-xii/problems/embedding-game                                 │
└──────────────────┴─────────────────────────────────────────────────────────────────────────────────────────────────────────┘

In [ ]:
!pip install pattern

     |████████████████████████████████| 22.2 MB 1.3 MB/s 
     |████████████████████████████████| 87 kB 6.0 MB/s 
     |████████████████████████████████| 81 kB 9.2 MB/s 
     |████████████████████████████████| 5.6 MB 48.5 MB/s 
     |████████████████████████████████| 5.6 MB 22.8 MB/s 
     |████████████████████████████████| 419 kB 43.7 MB/s 
     |████████████████████████████████| 97 kB 5.5 MB/s 
     |████████████████████████████████| 3.6 MB 28.0 MB/s 
  Created wheel for pattern: filename=Pattern-3.6-py3-none-any.whl size=22332721 sha256=1bf372bb2e2fceb1494cea9968ff850829d6f09b83260a4e01d715112b731392
  Stored in directory: /root/.cache/pip/wheels/8d/1f/4e/9b67afd2430d55dee90bd57618dd7d899f1323e5852c465682
  Created wheel for mysqlclient: filename=mysqlclient-2.1.0-cp37-cp37m-linux_x86_64.whl size=99964 sha256=f66e6dc09dde6cab51afad3cb4aad1f3f8f52d76aae65059abe8640d09a1bc40
  Stored in directory: /root/.cache/pip/wheels/97/d4/df/08cd6e1fa4a8691b268ab254bd0fa589827ab5b65638c010b4
  Cr

In [ ]:
from pattern.text.en import singularize

plurals = ['caresses', 'flies', 'dies', 'mules', 'geese', 'mice', 'bars', 'foos',
           'families', 'dogs', 'child', 'wolves']

singles = [singularize(plural) for plural in plurals]
print(singles)

['caress', 'fly', 'dy', 'mule', 'goose', 'mouse', 'bar', 'foo', 'family', 'dog', 'child', 'wolf']


In [ ]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec

model = Word2Vec(sentences=common_texts,size=100, window=5, min_count=1, workers=4)

In [ ]:
model.train([["hello", "world"]], total_examples=1, epochs=1)

(0, 2)

In [ ]:
import gensim.downloader

In [ ]:
gensim.downloader.info()['models'].keys()

dict_keys(['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis'])

In [ ]:
model = gensim.downloader.load('fasttext-wiki-news-subwords-300')

[==================================================] 100.0% 958.5/958.4MB downloaded


In [ ]:
model.most_similar('amongst')[0]

('among', 0.9080010652542114)

In [ ]:
rest_fill={}
for i in rest:
  print(i)
  try:
    p=model.most_similar(i)
    for j,k in p:
      if j in test_words:
        print(i+' -> '+j)
        rest_fill[i]=j
        break
  except:
    print(i+' not in dictionary')

equines
equines -> equine
remainder
remainder -> rest
amongst
amongst -> among
well
partnership
individuals
individuals -> people
subsidy
these
these -> many
vital
never
never -> not
thousanddollars
thousanddollars not in dictionary
losing
losing -> lost
frequently
frequently -> often
may
may -> might
several
several -> many
martin
complicated
complicated -> complex
havent
stalemate
stalemate -> deadlock
either
either -> or
oversubscribed
oversubscribed -> subscribed
private
private -> public
hundreds
hundreds -> thousands
below
below -> above
additon
additon -> addition
england
larger
larger -> large
firm
firm -> company
sold
sold -> bought
managership
managership -> management
grown
grown -> growing
themselves
four
four -> three
attendees
attendees -> participants
he
he -> she
ought
ought -> should
low
low -> high
calamitous
calamitous -> disastrous
were
were -> are
less
less -> more
paid
paid -> pay
made
made -> was
this
this -> the
similar
info
info -> information
repaid
repaid -> 

In [ ]:
for i in rest:
  if i not in rest_fill.keys():
    print(i)

well
partnership
subsidy
vital
thousanddollars
martin
havent
england
themselves
similar


In [ ]:
d['thousanddollars']=['thousands','dollars']

In [ ]:
d

{'bythe': ['the', 'by'],
 'fromthe': ['the', 'from'],
 'inthe': ['the', 'in'],
 'intothe': ['the', 'into'],
 'marketplace': ['place', 'market'],
 'onthe': ['the', 'on'],
 'thecapital': ['the', 'capital'],
 'thousanddollars': ['thousands', 'dollars'],
 'withinthe': ['the', 'within'],
 'withthe': ['the', 'with']}

In [ ]:
for i in rest:
  if i in d.keys():
    print(i)

In [ ]:
for i in rest:
  j=singularize(i)
  print(j)
  for k in test_words:
    kk=singularize(k)
    if kk==j:
      print(i+' -> '+k)

equine
equines -> equine
remainder
amongst
well
partnership
individual
subsidy
these
vital
never
thousanddollar
losing
frequently
may
several
martin
complicated
havent
stalemate
either
oversubscribed
private
hundred
below
additon
england
larger
firm
sold
managership
grown
themself
fmy
attendee
he
ought
low
calamitou
were
les
paid
made
thi
similar
info
repaid
had


In [ ]:
words=[]
for i in test_words:
  j=i.lower()
  words.append(singularize(j))
sorted(words)

['',
 'a',
 'above',
 'accounting',
 'accused',
 'action',
 'activity',
 'addition',
 'agreement',
 'allocate',
 'among',
 'an',
 'and',
 'any',
 'appraiser',
 'are',
 'as',
 'asset',
 'assist',
 'associated',
 'at',
 'banking',
 'be',
 'bought',
 'bubble',
 'busines',
 'business',
 'buy',
 'by',
 'can',
 'capital',
 'company',
 'complex',
 'consume',
 'contest',
 'contingent',
 'control',
 'could',
 'deadlock',
 'decision',
 'disastrou',
 'dispute',
 'divorce',
 'dollar',
 'drive',
 'economic',
 'economy',
 'equine',
 'establish',
 'estate',
 'estimating',
 'exchange',
 'farm',
 'finance',
 'for',
 'force',
 'formula',
 'friend',
 'from',
 'function',
 'fund',
 'fund',
 'further',
 'gambling',
 'general',
 'gift',
 'good',
 'government',
 'growing',
 'ha',
 'have',
 'hi',
 'high',
 'him',
 'horse',
 'horse',
 'in',
 'income',
 'increase',
 'industry',
 'inflation',
 'information',
 'initial',
 'interest',
 'into',
 'investment',
 'is',
 'large',
 'lawsuit',
 'legal',
 'lifeblood',
 'l

In [ ]:
dataset=pd.read_csv('data/sample_submission.csv')

In [ ]:
for i in dd.keys():
  try:
    p=model.most_similar(i)[0]
    print(i,dd[i])
    print(p)
  except:
    print(i)

exchanges ['exchange']
('exchange', 0.852319061756134)
was ['was']
('is', 0.8081117868423462)
systems ['system']
('system', 0.822524905204773)
decisions ['decision']
('decisons', 0.8226788640022278)
the from
longer ['longer']
('shorter', 0.755343496799469)
economy ['economy']
('economy-', 0.8162562251091003)
dispute ['disputes']
('disputes', 0.8630889654159546)
gifts ['gift']
('gift', 0.832480788230896)
the on
outlays ['outlay']
('expenditures', 0.8147125244140625)
further ['further']
('futher', 0.8049038648605347)
risks ['risk']
('risk', 0.8407328128814697)
the by
allocating ['allocate']
('reallocating', 0.8487706184387207)
actions ['action']
('inactions', 0.8178368806838989)
resolving ['resolve']
('solving', 0.7731807231903076)
when ['when']
('whenever', 0.7771919965744019)
assisting ['assist']
('aiding', 0.8545658588409424)
contingents ['contingent']
('contingent', 0.7521751523017883)
for ['for']
('time--for', 0.7398208975791931)
tool ['tools']
('tools', 0.7866291999816895)
the into

In [ ]:
Y=Y_new